# Ciencia de Datos en la Nube: El camino del "Azure ML SDK"

## Introducción

En este cuaderno, aprenderemos cómo usar el Azure ML SDK para entrenar, desplegar y consumir un modelo a través de Azure ML.

Requisitos previos:
1. Has creado un espacio de trabajo de Azure ML.
2. Has cargado el [conjunto de datos de Insuficiencia Cardíaca](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) en Azure ML.
3. Has subido este cuaderno a Azure ML Studio.

Los siguientes pasos son:

1. Crear un Experimento en un Espacio de Trabajo existente.
2. Crear un clúster de Computación.
3. Cargar el conjunto de datos.
4. Configurar AutoML usando AutoMLConfig.
5. Ejecutar el experimento de AutoML.
6. Explorar los resultados y obtener el mejor modelo.
7. Registrar el mejor modelo.
8. Desplegar el mejor modelo.
9. Consumir el endpoint.

## Importaciones específicas del SDK de Azure Machine Learning


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## Inicializar el Espacio de Trabajo
Inicializa un objeto de espacio de trabajo a partir de la configuración guardada. Asegúrate de que el archivo de configuración esté presente en .\config.json


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Crear un experimento de Azure ML

Vamos a crear un experimento llamado 'aml-experiment' en el espacio de trabajo que acabamos de inicializar.


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## Crear un Clúster de Cómputo
Necesitarás crear un [destino de cómputo](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) para tu ejecución de AutoML.


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## Datos
Asegúrate de haber subido el conjunto de datos a Azure ML y de que la clave tenga el mismo nombre que el conjunto de datos.


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

## Configuración de AutoML


In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

## Ejecución de AutoML


In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## Desplegar el Mejor Modelo

Ejecuta el siguiente código para desplegar el mejor modelo. Puedes ver el estado del despliegue en el portal de Azure ML. Este paso puede tardar unos minutos.


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## Consumir el Endpoint
Puedes agregar entradas al siguiente ejemplo de entrada.


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response


---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Si bien nos esforzamos por garantizar la precisión, tenga en cuenta que las traducciones automatizadas pueden contener errores o imprecisiones. El documento original en su idioma nativo debe considerarse la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que puedan surgir del uso de esta traducción.
